In [1]:
import numpy as np # numpy has various stat's helpers. Also contains functions to load files.
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures # we will use this to standardize the data
from sklearn.metrics import roc_auc_score # the metric we will be tested on . You can find more here :  https://www.kaggle.com/wiki/AreaUnderCurve
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold # the cross validation method we are going to use
import pandas as pd
import seaborn as sns
sns.set(style='ticks', palette='Set2')
import matplotlib
#from matplotlib import pyplot as plt
import matplotlib 
%matplotlib inline
import matplotlib.pyplot as plt
import itertools
import operator

RuntimeError: module compiled against API version a but this version of numpy is 9

/Users/shashank/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [12]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

In [3]:
def save_results(predictions,IDs,  filename):
    with open(filename, 'w') as f:
        f.write("id,Response\n")
        for i, pred in enumerate(predictions):
            f.write("%d,%f\n" % (IDs[i], pred))
#save_results(prediction, ID, filename)

In [4]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [ ]:
feature_columns_to_use=['var2','var4','var6','var9','var10','var11', 'var13', 'var15', 'var16', 'var17' ,'var20']
sns.pairplot(train, vars=feature_columns_to_use, diag_kind="kde", hue='response')

In [15]:
for i in test.columns:
    if i not in ['id']:
        print pd.DataFrame([pd.value_counts(train[i], sort=False)])#,pd.value_counts(train[i],sort=False)])

      0.00000  0.27666
var1    28795       15
         1      2     3     4    5    6     7
var2  1893  13518  4503  4017  381  338  4160
          0     1     2
var3  12322  7193  9295
      1.000000  0.474508  0.756788  0.738622  1.225309  1.185032  0.931966  \
var4       256      1064       792       905        16        23       409   

      0.959900  0.526081  0.986861    ...     0.500785  1.086289  0.574031  \
var4       337      1272       319    ...         1278        91      1386   

      1.074524  0.618884  0.872855  0.917611  1.326786  1.318063  1.235074  
var4       118      1395       578       489         7         4        11  

[1 rows x 65 columns]
      0.000000   3.571513   21.429075  14.286050  7.143025   17.857563  \
var5      14404      13436        137         18        396        385   

      10.714538  
var5         34  
      0.000000   13.636772  4.545591   9.091181 
var6      10638        389       3192      14591
      0   1      2    3    4
var7  1  25

In [ ]:
train.corr()

In [5]:
def parse_data(dtrain,dtest,target,ID,features=None):
    """Transform dataframe into numpy matrices"""
    if features is None:
        feature_columns_to_use = [column for column in dtrain.columns if column not in target]
    else:
        feature_columns_to_use = features

    X_trn = dtrain.as_matrix(feature_columns_to_use)
    y_trn = dtrain.as_matrix([target]).ravel()
    
    X_tst = dtest.as_matrix(feature_columns_to_use)
    y_id = dtest.as_matrix([ID]).ravel()
    
    return X_trn, y_trn, X_tst, y_id

def enumerate_columns(df,test,columns=None,drop_after_enumeration=False):
    if columns is None:
        columns_to_enumerate = df.columns
    else:
        columns_to_enumerate = columns
    if drop_after_enumeration == True:
        for column in columns_to_enumerate:
            print column
            enumerated_dict = {name : i for i, name in enumerate(np.unique(df[column]))}
            df['enumerated_'+column] = df[column].map(enumerated_dict).astype(int)
            df.drop(column,axis=1,inplace=True)
            
            test['enumerated_'+column] = test[column].map(enumerated_dict).astype(int)
            test.drop(column,axis=1,inplace=True)
        #return enumerated_dict
    else:
        for column in columns_to_enumerate:
            enumerated_dict = {name : i for i, name in enumerate(np.unique(df[column]))}
            df['enumerated_'+column] = df[column].map(enumerated_dict).astype(int) 
            
            test['enumerated_'+column] = test[column].map(enumerated_dict).astype(int)
        #return enumerated_dict
def sparsify(X, X_test):
    """Return One-Hot encoded datasets."""
    enc = preprocessing.OneHotEncoder()
    enc.fit(np.vstack((X, X_test)))
    return enc.transform(X), enc.transform(X_test)
def frequency(df,df_test,columns = None):
    listcol=[]
    if columns is None:
        columns_to_count = df.columns
    else:
        columns_to_count = columns
    for column in columns_to_count:
        count_dict=dict(df[column].value_counts())
        df["count_"+column]=df[column].map(count_dict)
        df_test["count_"+column]=df_test[column].map(count_dict)
        listcol.append("count_"+column)
    return listcol
#Value_Counts 2-way
def frequency_2_way(df, df_test, columns=None,drop_after=False,combinations=True):
    """Return 2-way counts"""
    listcol=[]
    if columns is None:
        columns_to_count = df.columns
    else:
        columns_to_count = columns
    if combinations == True:
        for i,j in itertools.combinations(columns_to_count,2):
            print "Counting %s and %s" %(i,j)
            frequency_dict=dict(df[[i,j]].groupby([i,j]).size())
            df[i+"_"+j]=pd.Series(zip(df[i],df[j])).map(frequency_dict)
            df_test[i+"_"+j]=pd.Series(zip(df_test[i],df_test[j])).map(frequency_dict)
            listcol.append(i+"_"+j)
    else:
        for (i,j) in columns_to_count:
            print "Counting %s and %s and %s" %(i,j)
            frequency_dict=dict(df[[i,j]].groupby([i,j]).size())
            df[i+"_"+j]=pd.Series(zip(df[i],df[j])).map(frequency_dict)
    if drop_after==True:
        if combinations==True:
            for column in columns_to_count:
                df.drop(column,axis=1,inplace=True)
        else:
            for column in set([element for tup in columns_to_count for element in tup]):
                df.drop(column,axis=1,inplace=True)
    return listcol
def frequency_3_way(df, df_test, columns=None,drop_after=False,combinations=True):
    """Return 3-way counts"""
    listcol=[]
    if columns is None:
        columns_to_count = df.columns
    else:
        columns_to_count = columns
    if combinations == True:
        for i,j,k in itertools.combinations(columns_to_count,3):
            print "Counting %s and %s and %s" %(i,j,k)
            frequency_dict=dict(df[[i,j,k]].groupby([i,j,k]).size())
            df[i+"_"+j+"_"+k]=pd.Series(zip(df[i],df[j],df[k])).map(frequency_dict)
            df_test[i+"_"+j+"_"+k]=pd.Series(zip(df_test[i],df_test[j],df_test[k])).map(frequency_dict)
            listcol.append(i+"_"+j+"_"+k)
    else:
        for (i,j,k) in columns_to_count:
            print "Counting %s and %s and %s" %(i,j,k)
            frequency_dict=dict(df[[i,j,k]].groupby([i,j,k]).size())
            df[i+"_"+j+"_"+k]=pd.Series(zip(df[i],df[j],df[k])).map(frequency_dict)
    if drop_after==True:
        if combinations==True:
            for column in columns_to_count:
                df.drop(column,axis=1,inplace=True)
        else:
            for column in set([element for tup in columns_to_count for element in tup]):
                df.drop(column,axis=1,inplace=True)

    return listcol


In [67]:
def greedy(df,target, columns=None, params=50, gap=0.0000099):
    train, test, y_t, y_test = cross_validation.train_test_split(df, df[target], test_size=0.2, random_state=0)
    #train=df
    scr = []
    if columns is None:
        columns=df.columns
    diff=0.1
    t=0
    while t<=params:
        score={}
        for f in columns:
            if f in scr:
                pass
            else:
                if scr == []:
                    past_score=0
                else:
                    past_score=temp
                scr.append(f)
                #print scr
                #scr.append(target)
                x_t = train[scr]
                x_test=test[scr]
                #x_t = x_t.dropna()
                #x_test = x_test.dropna()
                #y_t=x_t[target]
                #print y_t.shape
                if len(x_t.index) == 0:
                    scr.remove(f)
                    #scr.remove(target)
                    continue
                #x_t=x_t.drop([target],axis=1)
                #model = linear_model.LogisticRegression(C=0.2, class_weight='balanced', random_state=12, n_jobs=-1)
                #model.fit(x_t,y_t)
                #model=RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=10, min_samples_leaf=1, max_features=0.4, n_jobs=3, random_state=SEED) 
                #score[f]=cross_validation.cross_val_score(model,x_t,y_t,cv=5, scoring='roc_auc').mean()
                model = XGBClassifier(learning_rate =0.05, n_estimators=100, max_depth=4, min_child_weight=10, 
                      gamma=0.2, reg_alpha=0.8, reg_lambda=0.8, #subsample=0.9, colsample_bytree=1,
                      objective= 'binary:logistic', seed=12)
                #score[f]=cross_validation.cross_val_score(model,x_t,y_t,cv=5, scoring='roc_auc').mean()
                model.fit(x_t,y_t)
                score[f]=model.score(x_test,y_test)
                scr.remove(f)
                #scr.remove(target)
        sorted_score = sorted(score.items(), key=operator.itemgetter(1))
        #print sorted_score
        temp=max(score.iteritems(), key=operator.itemgetter(1))[1]
        if temp-past_score < gap:
            break
        else:
            added=max(score.iteritems(), key=operator.itemgetter(1))[0]
            print "HHHHH \n",added
            scr.append(added)
            print temp
            t=t+1
        print scr
    return scr

In [7]:
def blending(dtrain,dtest,clfs,clf=LogisticRegression(C=0.2),n_folds=10,verbose=True,shuffle=True,seed=0):
    np.random.seed(seed) # seed to shuffle the train set
    y = dtrain['response'].values
    skf = StratifiedKFold(y, n_folds)
    
    print "Creating train and test sets for blending."
    dataset_blend_train = np.zeros((dtrain.shape[0], len(clfs)))
    dataset_blend_test = np.zeros((dtest.shape[0], len(clfs)))
    for j, (feature_set,clf) in enumerate(clfs):
        print j, clf,feature_set
        X = dtrain[feature_set].values
        
        X_submission = dtest[feature_set].values
        dataset_blend_test_j = np.zeros((X_submission.shape[0], len(skf)))
        for i, (train, test) in enumerate(skf):
            print "Fold", i
            X_train = X[train]
            y_train = y[train]
            X_test = X[test]
            y_test = y[test]
            clf.fit(X_train, y_train)
            y_submission = clf.predict_proba(X_test)[:,1]
            dataset_blend_train[test, j] = y_submission
            dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:,1]
        dataset_blend_test[:,j] = dataset_blend_test_j.mean(1)

    print "Blending."
    clf.fit(dataset_blend_train, y)
    y_submission = clf.predict_proba(dataset_blend_test)[:,1]
    print 'Done'
    return y_submission

In [8]:
def interaction(df, df_test, columns = None):
    poly = PolynomialFeatures(2)
    if columns is None:
        columns_to_count = df.columns
    interac_train=pd.DataFrame(poly.fit_transform(df[columns]))
    #interac_train[target]=df[target]
    interac_test=pd.DataFrame(poly.fit_transform(df_test[columns]))
    return interac_train, interac_test

In [9]:

SEED=12
# === load data into numpy arrays === #
filename="my_name" # name prefix - you could put your name, like marios_mich
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
ID=test['id']
target='response'
train=train.drop(['id','var9','var1','var8'],axis=1)
test=test.drop(['id','var9','var1','var8'],axis=1)
#model = LogisticRegression(C=1)  # Sample of LogisticRegression model. We will adjust the C value base don our 5-fold CV
#model=RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=10, min_samples_leaf=1, max_features=0.4, n_jobs=3, random_state=SEED) 
model = XGBClassifier(learning_rate =0.05, n_estimators=300, max_depth=4, min_child_weight=20, 
                     gamma=0.2, reg_alpha=0.9, reg_lambda=0.9, subsample=0.9, colsample_bytree=1,
                     objective= 'binary:logistic', seed=12)

continuous=['var12','var18']
y_train=train[target]
extra_train=train[continuous]
extra_test=test[continuous]
predictors = [x for x in train.columns if (x not in target) and (x not in continuous)]
rows=train.shape[0]
train=train.drop(target,axis=1)
train=train.drop(continuous,axis=1)
test=test.drop(continuous,axis=1)
result = pd.concat([train,test])

#print "Enumerating Train and Test (on basis of Train)"
#enum_dict=enumerate_columns(train,test,columns=predictors,drop_after_enumeration=True)

predictors=train.columns
list1=frequency(result, test, columns=predictors)
train=result[:rows]
#train[continuous]=extra_train
train[target]=y_train
#test[continuous]=extra_test

# scaler=StandardScaler()
# train[continuous] = scaler.fit_transform(train[continuous])
# test[continuous] = scaler.fit_transform(test[continuous])



print "Making Interactions..."
interac_train, interac_test=interaction(train,test,columns=list1)
predic = interac_train.columns
interac_train[target]=y_train
#pred=greedy(interac_train, target, columns=predic)

print "2,3-way Counts Train, Test..."
list2=frequency_2_way(train,test, columns=predictors)
list3=frequency_3_way(train,test, columns=predictors)

#train=result[:rows]
train[continuous]=extra_train
train[target]=y_train
#test=result[rows:]
test[continuous]=extra_test
scaler=StandardScaler()
train[continuous] = scaler.fit_transform(train[continuous])
test[continuous] = scaler.fit_transform(test[continuous])

#predictors = [x for x in train.columns if (x not in target)]
#print "Greedy Selection..."
#preds=greedy(train,target, columns=predictors)
train[predic]=interac_train[predic]
test[predic]=interac_test[predic]




/Users/shashank/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Making Interactions...
2,3-way Counts Train, Test...
Counting var2 and var3
Counting var2 and var4


/Users/shashank/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Counting var2 and var5
Counting var2 and var6
Counting var2 and var7
Counting var2 and var10
Counting var2 and var11
Counting var2 and var13
Counting var2 and var14
Counting var2 and var15
Counting var2 and var16
Counting var2 and var17
Counting var2 and var19
Counting var2 and var20
Counting var2 and var21
Counting var3 and var4
Counting var3 and var5
Counting var3 and var6
Counting var3 and var7
Counting var3 and var10
Counting var3 and var11
Counting var3 and var13
Counting var3 and var14
Counting var3 and var15
Counting var3 and var16
Counting var3 and var17
Counting var3 and var19
Counting var3 and var20
Counting var3 and var21
Counting var4 and var5
Counting var4 and var6
Counting var4 and var7
Counting var4 and var10
Counting var4 and var11
Counting var4 and var13
Counting var4 and var14
Counting var4 and var15
Counting var4 and var16
Counting var4 and var17
Counting var4 and var19
Counting var4 and var20
Counting var4 and var21
Counting var5 and var6
Counting var5 and var7
Coun

/Users/shashank/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Counting var2 and var3 and var5
Counting var2 and var3 and var6
Counting var2 and var3 and var7
Counting var2 and var3 and var10
Counting var2 and var3 and var11
Counting var2 and var3 and var13
Counting var2 and var3 and var14
Counting var2 and var3 and var15
Counting var2 and var3 and var16
Counting var2 and var3 and var17
Counting var2 and var3 and var19
Counting var2 and var3 and var20
Counting var2 and var3 and var21
Counting var2 and var4 and var5
Counting var2 and var4 and var6
Counting var2 and var4 and var7
Counting var2 and var4 and var10
Counting var2 and var4 and var11
Counting var2 and var4 and var13
Counting var2 and var4 and var14
Counting var2 and var4 and var15
Counting var2 and var4 and var16
Counting var2 and var4 and var17
Counting var2 and var4 and var19
Counting var2 and var4 and var20
Counting var2 and var4 and var21
Counting var2 and var5 and var6
Counting var2 and var5 and var7
Counting var2 and var5 and var10
Counting var2 and var5 and var11
Counting var2 and 

/Users/shashank/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/frame.py:2320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/Users/shashank/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/shashank/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

NameError: name 'pred' is not defined

In [302]:
#train[target]=y
#train.info(verbose=True, null_counts=True)
preds=greedy(train,target, columns=predictors)
preds

['var10',
 'var2',
 'var21',
 'var15',
 'var6',
 'var4',
 'var19',
 'var5',
 'var7',
 'var17',
 'var20']

In [10]:
train[predic]=interac_train[predic]
test[predic]=interac_test[predic]

In [68]:
# tempo=predic
# predic=list(predic).append(target)
tempo=train[range(153)]
tempo[target]=train[target]
preds=greedy(tempo,target, columns=range(153))
preds

/Users/shashank/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


HHHHH 
102
0.732731690385
[102]
HHHHH 
25
0.742103436307
[102, 25]
HHHHH 
137
0.750086775425
[102, 25, 137]
HHHHH 
136
0.75199583478
[102, 25, 137, 136]
HHHHH 
92
0.75216938563
[102, 25, 137, 136, 92]
HHHHH 
109
0.752690038181
[102, 25, 137, 136, 92, 109]
HHHHH 
112
0.753557792433
[102, 25, 137, 136, 92, 109, 112]
HHHHH 
19
0.754425546685
[102, 25, 137, 136, 92, 109, 112, 19]
HHHHH 
37
0.754946199236
[102, 25, 137, 136, 92, 109, 112, 19, 37]
HHHHH 
2
0.755466851788
[102, 25, 137, 136, 92, 109, 112, 19, 37, 2]


[102, 25, 137, 136, 92, 109, 112, 19, 37, 2]

In [86]:
rf_pred=list(['var2','var3', 'var4','var5','var6',
              'var7','var10','var11','var13',
              'var14','var15','var16','var17','var19',
              'var20','var21','var18','var12'])
rf_pred

['var2',
 'var3',
 'var4',
 'var5',
 'var6',
 'var7',
 'var10',
 'var11',
 'var13',
 'var14',
 'var15',
 'var16',
 'var17',
 'var19',
 'var20',
 'var21',
 'var18',
 'var12']

In [52]:
model.fit(tempo[sc],tempo[target])
#tempo[target]
#train.info(verbose=True,null_counts=True)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.2, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=20, missing=None, n_estimators=300, nthread=-1,
       objective='binary:logistic', reg_alpha=0.9, reg_lambda=0.9,
       scale_pos_weight=1, seed=3, silent=True, subsample=0.9)

In [87]:
#preds=['var12','var18','enumerated_var10_enumerated_var13_enumerated_var17', 'enumerated_var13_enumerated_var15', 'enumerated_var2_enumerated_var3_enumerated_var13', 'enumerated_var11_enumerated_var13_enumerated_var17', 'enumerated_var13_enumerated_var20_enumerated_var21']
#new=train[preds]
#new[target]=train[target]
#test.fillna(0)
#test['id']=ID
#X, _, X_test,_=parse_data(train,test,target,'id')
ids=ID
#y=new[target]
#train[target]=y
#train.drop(target,axis=1, inplace=True)

#X, X_test = sparsify(X, X_test)
# === load data into numpy arrays === #
# print ("loading training data")
# X=np.loadtxt("train.csv",skiprows=1, delimiter=",", usecols=[i for i in range (1, 22)]) # 1 is inclusive and 22 exclusive. so it is [1,21] basically. We skip 1 row since the first one is headers
# print ("loading labels")    
# y=np.loadtxt("train.csv",skiprows=1, delimiter=",", usecols=[22]) # The last one - the respnse variable   
# print ("loading test data")
# X_test=np.loadtxt("test.csv",skiprows=1, delimiter=",", usecols=[i for i in range (1, 22)]) # 1 is inclusive and 22 exclusive. so it is [1,21] basically
# print ("loading ids")    
# ids=np.loadtxt("test.csv",skiprows=1, delimiter=",", usecols=[0]) # The first column is the id
rf_model=RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=10, min_samples_leaf=1, max_features=0.4, n_jobs=3, random_state=SEED) 
model = XGBClassifier(learning_rate =0.05, n_estimators=300, max_depth=4, min_child_weight=20, 
                     gamma=0.2, reg_alpha=0.9, reg_lambda=0.9, subsample=0.9, colsample_bytree=1,
                     objective= 'binary:logistic', seed=3)
model2 = XGBClassifier(learning_rate =0.06, n_estimators=100, max_depth=5, min_child_weight=20, 
                     gamma=0.2, reg_alpha=0.9, reg_lambda=0.9, subsample=0.9, colsample_bytree=1,
                     objective= 'binary:logistic', seed=3)
clfs=[(rf_pred,rf_model),(preds,model), (pred,model2)]
# === training & metrics === #
mean_auc = 0.0
n = 5  # number of folds in strattified cv
kfolder=StratifiedKFold(y, n_folds= n,shuffle=True, random_state=12)     
i=0
for train_index, test_index in kfolder: # for each train and test pair of indices in the kfolder object
    # creaning and validation sets
#     X_train, X_cv = X[train_index], X[test_index]
#     y_train, y_cv = np.array(y)[train_index], np.array(y)[test_index]
    X_train, X_cv=train.iloc[train_index], train.iloc[test_index]
    y_train, y_cv=y.iloc[train_index], y.iloc[test_index]
    print (" train size: %d. test size: %d, cols: %d " % ((X_train.shape[0]) ,(X_cv.shape[0]) ,(X_train.shape[1]) ))

    # do scalling
#     scaler=StandardScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_cv = scaler.transform(X_cv)

    # train model
    #model.fit(X_train,y_train)
    # make predictions in probabilities
    #prediction=model.predict_proba(X_cv)[:,1]
    # compute AUC metric for this CV fold
    
    prediction=blending(X_train,X_cv,clfs)
    roc_auc = roc_auc_score(y_cv, prediction)
    print "AUC (fold %d/%d): %f" % (i + 1, n, roc_auc)
    mean_auc += roc_auc
    i+=1

mean_auc/=n
print (" Average AUC: %f" % (mean_auc) )        

# do scalling
# scaler=StandardScaler()
# X = scaler.fit_transform(X)
# X_test = scaler.transform(X_test) 



 train size: 23048. test size: 5762, cols: 868 
Creating train and test sets for blending.
0 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=0.4, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=3,
            oob_score=False, random_state=12, verbose=0, warm_start=False) ['var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var10', 'var11', 'var13', 'var14', 'var15', 'var16', 'var17', 'var19', 'var20', 'var21', 'var18', 'var12']
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
1 XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.2, learning_rate=0.05, max_delta_step=0, max_depth=4,
       min_child_weight=20, missing=None, n_estimators=300, nthread=-1,
       objective='binary:logistic', reg_alpha=0.9, reg_lambda=0.9,
       scale_pos_weight=1, seed=3, silent=True, subsample=0.

In [88]:
# new, cv, y_t, y_cv = cross_validation.train_test_split(train, train[target], test_size=0.2, random_state=0)
# new[target]=y_t

#rf_mod=RandomForestClassifier(n_estimators=300, criterion='entropy', max_depth=10, min_samples_leaf=1, max_features=0.4, n_jobs=3, random_state=SEED) 
#ett_mod=ExtraTreesClassifier(n_estimators=300, n_jobs=-1, criterion='entropy')
#gbm=GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=10, n_estimators=100)
#clfs=[(preds,model), (rf_pred,rf_mod)]
prediction=blending(train,test,clfs)

Creating train and test sets for blending.
0 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=0.4, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=3,
            oob_score=False, random_state=12, verbose=0, warm_start=False) ['var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var10', 'var11', 'var13', 'var14', 'var15', 'var16', 'var17', 'var19', 'var20', 'var21', 'var18', 'var12']
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
1 XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.2, learning_rate=0.05, max_delta_step=0, max_depth=4,
       min_child_weight=20, missing=None, n_estimators=300, nthread=-1,
       objective='binary:logistic', reg_alpha=0.9, reg_lambda=0.9,
       scale_pos_weight=1, seed=3, silent=True, subsample=0.9) [102, 25, 137, 136, 92, 109, 112, 19, 37, 2]


In [89]:
#make final model
#X=new[preds]
#model.fit(train,y)

#make predictions in probabilities
#print roc_auc_score(y_cv, prediction)

#prediction=model.predict_proba(test)[:,1]    
#create submission file 
save_results(prediction, ids,  filename+"_submission_" +str(mean_auc).replace(".","_") + ".csv") # putting the actuall AUC (of cv) in the file's name for reference

print("Submission %s has been generated!" % ( filename+"_submission_" +str(mean_auc).replace(".","_") + ".csv"))

Submission my_name_submission_0_825363568067.csv has been generated!


In [289]:
len(preds)
#test.info(verbose=True)

20

In [81]:
y.shape

(28810,)

In [90]:
temporary.remove('var12')
temporary.remove('var18')

In [119]:
temporary

['enumerated_var10_enumerated_var13_enumerated_var17',
 'enumerated_var13_enumerated_var15',
 'enumerated_var2_enumerated_var3_enumerated_var13',
 'enumerated_var11_enumerated_var13_enumerated_var17',
 'enumerated_var13_enumerated_var20_enumerated_var21',
 'enumerated_var13_enumerated_var14_enumerated_var21',
 'enumerated_var5_enumerated_var13_enumerated_var20',
 'enumerated_var13_enumerated_var15_enumerated_var17',
 'enumerated_var2_enumerated_var4_enumerated_var21',
 'var12',
 'var18',
 'var12',
 'var18']

In [120]:
preds

In [99]:
test=test.fillna(0)

In [2]:
from __future__ import division
from collections import defaultdict
from glob import glob
import sys
import math
import os
loc_outfile = 'out.csv'
def kaggle_bag(array, loc_outfile, weights=[0.8,0.2],method="average"):
  if method == "average":
    scores = defaultdict(float)
  with open(loc_outfile,"wb") as outfile:
    for i, glob_file in enumerate(array):
      print "parsing:", i,glob_file
      # sort glob_file by first column, ignoring the first line
      lines = open(glob_file).readlines()
      lines = [lines[0]] + sorted(lines[1:])
      for e, line in enumerate( lines ):
        if i == 0 and e == 0:
          outfile.write(line)
        if e > 0:
          row = line.strip().split(",")
          scores[(e,row[0])] += float(row[1])*weights[i]
    for j,k in sorted(scores):
      outfile.write("%s,%f\n"%(k,(scores[(j,k)])))
    print("wrote to %s"%loc_outfile)
array=[ os.getcwd()+'/83505.csv', os.getcwd()+'/83255.csv']
kaggle_bag(array, loc_outfile)

parsing: 0 /Users/shashank/Desktop/MS/1-2/ML/Midterm/83505.csv
parsing: 1 /Users/shashank/Desktop/MS/1-2/ML/Midterm/83255.csv
wrote to out.csv


In [263]:
# temporary.remove('var12')
# temporary.remove('var18')
#Phani chosen parameters
#preds=['enumerated_var10','var12','var18','enumerated_var21','enumerated_var15','enumerated_var6','enumerated_var2','enumerated_var4','enumerated_var3','enumerated_var7','enumerated_var16','enumerated_var5',]
#Greedy selected on Logistic 
'''
preds_top_3=
['enumerated_var10_enumerated_var13_enumerated_var17',
 'enumerated_var2_enumerated_var3_enumerated_var13',
 'enumerated_var11_enumerated_var13_enumerated_var17',
 'enumerated_var13_enumerated_var20_enumerated_var21',
 'enumerated_var13_enumerated_var14_enumerated_var21',
 'enumerated_var5_enumerated_var13_enumerated_var20',
 'enumerated_var13_enumerated_var15_enumerated_var17',
 'enumerated_var2_enumerated_var4_enumerated_var21'
 ]
pred_top_2=
['enumerated_var3_enumerated_var10', 
 'enumerated_var3_enumerated_var15',
 'enumerated_var2_enumerated_var6',
 'enumerated_var19_enumerated_var21',
 'enumerated_var10_enumerated_var15',
 'enumerated_var3_enumerated_var20'
]
preds_single=
['enumerated_var10', 
 'enumerated_var15', 
 'enumerated_var2', 
 'enumerated_var21', 
 'enumerated_var13', 
 'var18', 
 'var12',
 'enumerated_var19',
 'enumerated_var20',
 'enumerated_var11',
 'enumerated_var7'
]
'''
preds=['enumerated_var10_enumerated_var13_enumerated_var17',
 'enumerated_var2_enumerated_var3_enumerated_var13',
 'enumerated_var11_enumerated_var13_enumerated_var17',
 'enumerated_var13_enumerated_var20_enumerated_var21',
 'enumerated_var13_enumerated_var14_enumerated_var21',
 'enumerated_var5_enumerated_var13_enumerated_var20',
 'enumerated_var13_enumerated_var15_enumerated_var17',
 'enumerated_var2_enumerated_var4_enumerated_var21',
 'enumerated_var3_enumerated_var10', 
 #'enumerated_var3_enumerated_var15',
 #'enumerated_var2_enumerated_var6',
 #'enumerated_var19_enumerated_var21',
 #'enumerated_var10_enumerated_var15',
 #'enumerated_var3_enumerated_var20',
 'enumerated_var10', 
 'enumerated_var15', 
 'enumerated_var2', 
 'enumerated_var21', 
 'enumerated_var13', 
 'var18', 
 'var12',
 'enumerated_var19',
 'enumerated_var20',
 'enumerated_var11',
 'enumerated_var7'
]
preds

['enumerated_var10_enumerated_var13_enumerated_var17',
 'enumerated_var2_enumerated_var3_enumerated_var13',
 'enumerated_var11_enumerated_var13_enumerated_var17',
 'enumerated_var13_enumerated_var20_enumerated_var21',
 'enumerated_var13_enumerated_var14_enumerated_var21',
 'enumerated_var5_enumerated_var13_enumerated_var20',
 'enumerated_var13_enumerated_var15_enumerated_var17',
 'enumerated_var2_enumerated_var4_enumerated_var21',
 'enumerated_var3_enumerated_var10',
 'enumerated_var10',
 'enumerated_var15',
 'enumerated_var2',
 'enumerated_var21',
 'enumerated_var13',
 'var18',
 'var12',
 'enumerated_var19',
 'enumerated_var20',
 'enumerated_var11',
 'enumerated_var7']

In [28]:
pred=[7, 11, 21, 13, 20]

KeyError: 167

In [242]:
len(preds)

20

In [250]:
len(pred)

8

In [72]:
model = XGBClassifier(learning_rate =0.06, 
                      n_estimators=100, 
                      max_depth=5, 
                      min_child_weight=20, 
                     gamma=0.2, 
                      reg_alpha=0.9, 
                      reg_lambda=0.9, 
                      subsample=0.9, 
                      colsample_bytree=1,
                     objective= 'binary:logistic', 
                      seed=3)
clf=RandomizedSearchCV(model,
    {
        #'learning_rate':[i/100.0 for i in range(5,20)],
         #'min_child_weight': [5,6,7,8],
         #'max_depth': [5,10,15,18,12,50],
         'n_estimators':[20,40,70,100,200,300,350,250,320] ,
        'gamma':[i/10.0 for i in range(2,4)],
        #'reg_alpha':[0.2,0.3,0.4,0.5],
        #'reg_lambda':[0.3,0.4,0.5,0.6]
        #'colsample_bytree': [0.6,0.7,0.8,0.85,0.9],
    },
    cv=5,
    n_jobs=1,
    verbose=5,
    scoring='roc_auc'
)
clf

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.2, learning_rate=0.06, max_delta_step=0, max_depth=5,
       min_child_weight=20, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0.9, reg_lambda=0.9,
       scale_pos_weight=1, seed=3, silent=True, subsample=0.9),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': [20, 40, 70, 100, 200, 300, 350, 250, 320], 'gamma': [0.2, 0.3]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='roc_auc', verbose=5)

In [73]:
clf.fit(train[pred],train[target])

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=250, gamma=0.2 .....................................
[CV] ............ n_estimators=250, gamma=0.2, score=0.810951 -   2.4s
[CV] n_estimators=250, gamma=0.2 .....................................
[CV] ............ n_estimators=250, gamma=0.2, score=0.810734 -   2.3s
[CV] n_estimators=250, gamma=0.2 .....................................
[CV] ............ n_estimators=250, gamma=0.2, score=0.815581 -   2.2s
[CV] n_estimators=250, gamma=0.2 .....................................
[CV] ............ n_estimators=250, gamma=0.2, score=0.807219 -   2.2s
[CV] n_estimators=250, gamma=0.2 .....................................
[CV] ............ n_estimators=250, gamma=0.2, score=0.810762 -   2.2s
[CV] n_estimators=40, gamma=0.2 ......................................
[CV] ............. n_estimators=40, gamma=0.2, score=0.805275 -   0.4s
[CV] n_estimators=40, gamma=0.2 ......................................
[CV] ...........

[Parallel(n_jobs=1)]: Done  17 tasks       | elapsed:   30.8s


[CV] ............ n_estimators=350, gamma=0.3, score=0.814390 -   3.6s
[CV] n_estimators=350, gamma=0.3 .....................................
[CV] ............ n_estimators=350, gamma=0.3, score=0.807067 -   4.0s
[CV] n_estimators=350, gamma=0.3 .....................................
[CV] ............ n_estimators=350, gamma=0.3, score=0.809168 -   3.7s
[CV] n_estimators=320, gamma=0.2 .....................................
[CV] ............ n_estimators=320, gamma=0.2, score=0.810543 -   3.3s
[CV] n_estimators=320, gamma=0.2 .....................................
[CV] ............ n_estimators=320, gamma=0.2, score=0.809924 -   3.0s
[CV] n_estimators=320, gamma=0.2 .....................................
[CV] ............ n_estimators=320, gamma=0.2, score=0.814921 -   3.5s
[CV] n_estimators=320, gamma=0.2 .....................................
[CV] ............ n_estimators=320, gamma=0.2, score=0.806815 -   2.9s
[CV] n_estimators=320, gamma=0.2 .....................................
[CV] .

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.8min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.2, learning_rate=0.06, max_delta_step=0, max_depth=5,
       min_child_weight=20, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0.9, reg_lambda=0.9,
       scale_pos_weight=1, seed=3, silent=True, subsample=0.9),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': [20, 40, 70, 100, 200, 300, 350, 250, 320], 'gamma': [0.2, 0.3]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='roc_auc', verbose=5)

In [74]:
print(clf.best_params_) 
print(clf.best_score_)

{'n_estimators': 100, 'gamma': 0.2}
0.811674616512


In [69]:
pred=[102, 25, 137, 136, 92, 109, 112, 19, 37, 2]